In [1]:
import logging
import sys

import numpy as np
import tvm
from tvm import te
import tvm.testing

# 模块名叫 `autotvm`
from tvm import autotvm

In [2]:
def matmul_basic(N, L, M, dtype):

    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, M), name="B", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name="C")
    s = te.create_schedule(C.op)

    # 调度
    y, x = s[C].op.axis
    k = s[C].op.reduce_axis[0]

    yo, yi = s[C].split(y, 8)
    xo, xi = s[C].split(x, 8)

    s[C].reorder(yo, xo, k, yi, xi)

    return s, [A, B, C]

In [3]:
s, [A, B, C] = matmul_basic(10, 10, 10, "float32")

In [4]:
print(tvm.lower(s, [A, B, C]))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((10, 10), "float32"), B: T.Buffer((10, 10), "float32"), C: T.Buffer((10, 10), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        for i_outer, j_outer in T.grid(2, 2):
            C_1 = T.Buffer((100,), data=C.data)
            for i_inner_init in range(8):
                if i_outer * 4 + i_inner_init // 2 < 5:
                    for j_inner_init in range(8):
                        if j_outer * 4 + j_inner_init // 2 < 5:
                            C_1[i_outer * 80 + i_inner_init * 10 + j_outer * 8 + j_inner_init] = T.float32(0)
            for k, i_inner in T.grid(10, 8):
                if i_outer * 4 + i_inner // 2 < 5:
                    for j_inner in range(8):
                        if j_outer * 4 + j_inner // 2 < 5:
                            cse_var_3: T.int32 = j_outer * 8
   